In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from utils import *

In [4]:
# Split the DataFrame df into features (X) and target (y)
data=pd.read_csv('data/training.csv')
df_data = data.drop(columns=['EventId','Weight'])
X = df_data.drop(columns=['Label'])  # Features

X = np.asarray(pd.get_dummies(X, columns=['PRI_jet_num'], prefix='PRI_jet_num').values, dtype = np.float32)
X = X/np.linalg.norm(X, axis=1).max()
y = df_data['Label'].values  # Target
y = np.where(y == 's', 1, 0)
y = np.asarray(y, dtype = np.float32)




In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
import torch

# Convert X_train and X_test to tensors
x_train_tensor = torch.tensor(X_train)
x_test_tensor = torch.tensor(X_test)

# Convert y_train and y_test to tensors
y_train_tensor = torch.tensor(y_train)
y_test_tensor = torch.tensor(y_test)
# x_train_tensor = normalize2(x_train_tensor)
# x_test_tensor = normalize2(x_test_tensor)

In [11]:
print("\n \n** Formed Tensors and starting model training** \n \n ")
net = Net_new(1,device = torch.device('cuda:3'))
trainloader = torch.utils.data.DataLoader(list(zip(x_train_tensor, y_train_tensor)), batch_size=4096, shuffle=False)

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
net = net.to(device=torch.device('cuda:3'))

train_model_priv(net,trainloader,x_test_tensor,y_test_tensor,optimizer,1,0.8,device= torch.device('cuda:3'),print_cond = True,only_reg_flag=0,lr_schedular =None,lambda_loss=1)
outputs = net(x_test_tensor)
print(((outputs>0.5).squeeze().cpu() == y_test_tensor.squeeze()).sum()/(len(y_test_tensor)))


 
** Formed Tensors and starting model training** 
 
 
tensor(1.6581) tensor(0.9698)
tensor(1.5434) tensor(0.8588)
tensor(1.5184) tensor(0.8377)
tensor(1.4996) tensor(0.8230)
tensor(1.4698) tensor(0.7937)
tensor(1.4652) tensor(0.7915)
tensor(1.4526) tensor(0.7852)
tensor(1.4516) tensor(0.7839)
tensor(1.4383) tensor(0.7763)
tensor(1.4285) tensor(0.7689)
tensor(1.4332) tensor(0.7752)
tensor(1.4260) tensor(0.7676)
tensor(1.4240) tensor(0.7713)
tensor(1.4252) tensor(0.7761)
tensor(1.4149) tensor(0.7697)
tensor(1.4045) tensor(0.7650)
tensor(1.4206) tensor(0.7728)
tensor(1.4030) tensor(0.7650)
tensor(1.4078) tensor(0.7660)
tensor(1.4081) tensor(0.7756)
tensor(1.3994) tensor(0.7636)
tensor(1.4095) tensor(0.7728)
tensor(1.3955) tensor(0.7690)
tensor(1.4025) tensor(0.7696)
tensor(1.3898) tensor(0.7640)
tensor(1.4016) tensor(0.7689)
tensor(1.3921) tensor(0.7585)
tensor(1.3826) tensor(0.7633)
tensor(1.3985) tensor(0.7662)
tensor(1.3994) tensor(0.7709)
tensor(1.3900) tensor(0.7587)
tensor(1.3950)

In [9]:
x_train_tensor[0]

tensor([ 3.2395e-02,  3.3172e-03,  1.8058e-02,  4.8197e-02,  4.3333e-04,
         6.0725e-02,  1.2431e-04,  2.8626e-04,  1.6094e-03,  8.5882e-02,
         2.2279e-04,  2.5934e-04,  7.3754e-05,  1.4200e-02,  9.2356e-05,
         2.2454e-04,  1.4453e-02, -4.6178e-05,  4.7491e-04,  2.5048e-02,
         4.3661e-04,  9.9252e-02,  4.5627e-02, -5.5589e-05, -2.4643e-04,
         1.1603e-02, -4.8892e-04, -5.4013e-04,  5.7230e-02,  0.0000e+00,
         0.0000e+00,  2.1885e-04,  0.0000e+00])

In [10]:
x_test_tensor[0]

tensor([ 2.3266e-02,  5.4836e-03,  1.6235e-02,  4.0966e-03, -1.7044e-01,
        -1.7044e-01, -1.7044e-01,  4.5211e-04,  4.0966e-03,  1.3821e-02,
         8.5133e-05, -1.6651e-04, -1.7044e-01,  9.2203e-03, -5.1182e-06,
        -2.5011e-04,  4.6011e-03, -2.7928e-04,  1.0936e-04,  6.9328e-03,
         2.8235e-04,  2.9274e-02, -1.7044e-01, -1.7044e-01, -1.7044e-01,
        -1.7044e-01, -1.7044e-01, -1.7044e-01,  0.0000e+00,  1.7061e-04,
         0.0000e+00,  0.0000e+00,  0.0000e+00], grad_fn=<SelectBackward0>)

In [12]:
print("\n \n ** Creating embeddings ** \n \n ",len(y_train_tensor))
trainloader = torch.utils.data.DataLoader(list(zip(x_train_tensor, y_train_tensor)), batch_size=4096, shuffle=False)
X_emb_train, losses_train = create_model_embs2(net,trainloader,device= torch.device('cuda:3'),l=len(y_train_tensor),h=0.8)





 
 ** Creating embeddings ** 
 
  200000


In [13]:
# x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
# x_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
x_test_tensor = torch.tensor(X_test)
# x_test_tensor = normalize2(x_test_tensor)

# y_test_tensor = torch.tensor(y_test, dtype=torch.float32)
testloader = torch.utils.data.DataLoader(list(zip(x_test_tensor, y_test_tensor)), batch_size=4096, shuffle=False)
X_emb_test, losses_test = create_model_embs2(net,testloader,device= torch.device('cuda:3'),l=len(y_test_tensor),h=0.8)
losses_train,indices = torch.sort(losses_train)

# x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
# x_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

# y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [14]:

set_eps = 1
ind = (losses_train < set_eps).sum()

    


X_emb_train_priv = X_emb_train[indices][:ind]
Y_train = y_train_tensor[indices][:ind]



In [15]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=20)

# Train the model
model.fit(X_emb_train_priv, Y_train)

# Make predictions
predictions = model.predict(X_emb_test)
(predictions == np.array(y_test_tensor)).sum()/(len(y_test_tensor))

0.73322

In [17]:
num_epochs = 20
model = nn.Sequential(
    nn.Linear(33, 64),  # Input layer with 100 input features and 64 output features
    nn.ReLU(),  # Activation function
    nn.Linear(64, 128),  # Hidden layer with 64 input features and 32 output features
    nn.ReLU(), 
    nn.Linear(128,32),  # Hidden layer with 64 input features and 32 output features
    nn.ReLU(),# Activation function
    nn.Linear(32, 1),
    nn.Sigmoid()# Output layer with 32 input features and 10 output features
)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=0.00001)
trainloader_priv = torch.utils.data.DataLoader(list(zip(X_emb_train_priv, Y_train)), batch_size=512, shuffle=True)
print("\n \n ** Starting final training ** \n \n ")
train_emb(model,trainloader_priv,X_emb_test,y_test_tensor,nn.BCELoss(),optimizer,num_epochs,device=torch.device('cuda:0'),test_total_loader = None,max_steps =10000)


 
 ** Starting final training ** 
 
 
Test Accuracy:  tensor(0.6613)
Test Accuracy:  tensor(0.6613)
Test Accuracy:  tensor(0.6613)
Test Accuracy:  tensor(0.6592)
Test Accuracy:  tensor(0.6677)
Test Accuracy:  tensor(0.6919)
Test Accuracy:  tensor(0.7068)
Test Accuracy:  tensor(0.7095)
Test Accuracy:  tensor(0.7101)
Test Accuracy:  tensor(0.7143)
Test Accuracy:  tensor(0.7169)
Test Accuracy:  tensor(0.7182)
Test Accuracy:  tensor(0.7220)
Test Accuracy:  tensor(0.7239)
Test Accuracy:  tensor(0.7235)
Test Accuracy:  tensor(0.7237)
Test Accuracy:  tensor(0.7177)
Test Accuracy:  tensor(0.7222)
Test Accuracy:  tensor(0.7282)
Test Accuracy:  tensor(0.7313)


In [18]:
input_size = 33
num_epochs = 50
# model = LogisticRegression(input_size)
model = nn.Sequential(
    nn.Linear(33, 64),  # Input layer with 100 input features and 64 output features
    nn.ReLU(),  # Activation function
    nn.Linear(64, 128),  # Hidden layer with 64 input features and 32 output features
    nn.ReLU(), 
    nn.Linear(128,32),  # Hidden layer with 64 input features and 32 output features
    nn.ReLU(),# Activation function
    nn.Linear(32, 1),
    nn.Sigmoid()# Output layer with 32 input features and 10 output features
)
from opacus import PrivacyEngine
privacy_engine = PrivacyEngine()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model2, optimizer2, data_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=trainloader,
    target_epsilon=1,
    target_delta =0.0001,
    epochs = num_epochs,
    max_grad_norm=1.0,
)
train_emb(model2,data_loader,x_test_tensor,y_test_tensor,nn.BCELoss(),optimizer2,num_epochs,device=torch.device('cuda:0'),max_steps =10000)

/raid/ganesh/racha_suraj/miniconda3/envs/dpo/lib/python3.12/site-packages/opacus/privacy_engine.py:95: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/raid/ganesh/racha_suraj/miniconda3/envs/dpo/lib/python3.12/site-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/raid/ganesh/racha_suraj/miniconda3/envs/dpo/lib/python3.12/site-packages/torch/nn/modules/module.py:1373: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Usi

Test Accuracy:  tensor(0.6613)
Test Accuracy:  tensor(0.6613)
Test Accuracy:  tensor(0.6613)
Test Accuracy:  tensor(0.6613)
Test Accuracy:  tensor(0.6613)
Test Accuracy:  tensor(0.6613)
Test Accuracy:  tensor(0.6613)
Test Accuracy:  tensor(0.6613)
Test Accuracy:  tensor(0.6615)
Test Accuracy:  tensor(0.6639)
Test Accuracy:  tensor(0.6664)
Test Accuracy:  tensor(0.6691)
Test Accuracy:  tensor(0.6717)
Test Accuracy:  tensor(0.6760)
Test Accuracy:  tensor(0.6786)
Test Accuracy:  tensor(0.6812)
Test Accuracy:  tensor(0.6839)
Test Accuracy:  tensor(0.6866)
Test Accuracy:  tensor(0.6884)
Test Accuracy:  tensor(0.6904)
Test Accuracy:  tensor(0.6912)
Test Accuracy:  tensor(0.6917)
Test Accuracy:  tensor(0.6926)
Test Accuracy:  tensor(0.6930)
Test Accuracy:  tensor(0.6933)
Test Accuracy:  tensor(0.6935)
Test Accuracy:  tensor(0.6937)
Test Accuracy:  tensor(0.6940)
Test Accuracy:  tensor(0.6943)
Test Accuracy:  tensor(0.6944)
Test Accuracy:  tensor(0.6945)
Test Accuracy:  tensor(0.6948)
Test Acc

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


model = XGBClassifier()

model.fit(X_emb_train_priv, Y_train)

# Make predictions
predictions = model.predict(X_emb_test)

print("Accuracy:", (predictions == np.array(y_test_tensor)).sum()/(len(y_test_tensor)))

Accuracy: 0.71584


In [20]:
model = RandomForestClassifier(n_estimators=20)

# Train the model
model.fit(x_train_tensor, y_train_tensor)

# Make predictions
predictions = model.predict(x_test_tensor)
(predictions == np.array(y_test_tensor)).sum()/(len(y_test_tensor))

0.83132

In [21]:
model = XGBClassifier()

model.fit(x_train_tensor, y_train_tensor)

# Make predictions
predictions = model.predict(x_test_tensor)
(predictions == np.array(y_test_tensor)).sum()/(len(y_test_tensor))

0.83946